# Pendahuluan
- Langchain's Sales GPT

# Pendahuluan
Modul ini dirancang untuk memberikan praktik hands-on dalam prompting untuk aplikasi telemarketing-bot. Modul ini tidak menggunakan LangChain sebagai orkestrator agent, melainkan berfokus pada teknik prompting yang efektif untuk menghasilkan percakapan yang natural dan engaging.

Tersedia _dropdown_ menu untuk memilih LLM yang digunakan baik Open AI maupun Hugging Face.

Pada bagian akhir, agent-AI akan dibungkus ke dalam antarmuka _chatbot_ berbasis _web_ menggunakan Gradio.

In [ ]:
!pip install -q --upgrade openai
!pip install -q transformers accelerate datasets evaluate
!pip -q install --upgrade -i https://pypi.org/simple/ bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cpu


In [ ]:
from google.colab import userdata
from openai import OpenAI
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

LLM_Agent_Type = "Open AI" # @param ["TinyLlama", "Zephyr", "Sea Lion", "Open AI"]

def create_hf_gen(model, tokenizer):
  def hf_gen(prompt):
    messages = [
        {"role": "user", "content": prompt},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
    gen_tokens = model.generate(
        input_ids,
        do_sample=True,
        temperature=0.9,
        max_length=2048,
    )
    gen_text = tokenizer.batch_decode(gen_tokens[:, input_ids.shape[1]:], skip_special_tokens=True)[0]
    return gen_text

  return hf_gen


llm_gen = None
match LLM_Agent_Type:

  case "Open AI":
    client = OpenAI(
        api_key=userdata.get("OAI_KEY")
    )

    def oai_gen(prompt):
      completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
          {"role": "user", "content": prompt}
        ]
      )
      return completion.choices[0].message.content

    llm_gen = oai_gen

  case "TinyLlama":
    model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
    model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    llm_gen = create_hf_gen(model, tokenizer)

  case "Zephyr":
    model_name = "HuggingFaceH4/zephyr-7b-alpha"
    quantization_config = BitsAndBytesConfig(
      load_in_8bit=True
    )
    model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config, device_map=device)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    llm_gen = create_hf_gen(model, tokenizer)

  case "Sea Lion":
    model_name = "aisingapore/sea-lion-7b-instruct"
    quantization_config = BitsAndBytesConfig(
      load_in_8bit=True
    )
    model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=quantization_config, device_map=device)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    llm_gen = create_hf_gen(model, tokenizer)

In [ ]:
import jinja2

SALES_PROMPT_TEMPLATE = """Never forget your name is {{ salesperson_name }}. You work as a {{ salesperson_role }}.
You work at company named {{ company_name }}. {{ company_name }}'s business is the following: {{ company_business }}.
Company values are the following. {{ company_values }}
You are contacting a potential prospect in order to {{ conversation_purpose }}
Your means of contacting the prospect is {{ conversation_type }}.

If you're asked about where you got the user's contact information, say that you got it from public records.
Keep your responses in short length to retain the user's attention. Never produce lists, just answers.
Start the conversation by just a greeting and how is the prospect doing without pitching in your first turn.
When the conversation is over, output <END_OF_CALL>
Always think about at which conversation stage you are at before answering:

1: Introduction: Start the conversation by introducing yourself and your company. Be polite and respectful while keeping the tone of the conversation professional. Your greeting should be welcoming. Always clarify in your greeting the reason why you are calling.
2: Qualification: Qualify the prospect by confirming if they are the right person to talk to regarding your product/service. Ensure that they have the authority to make purchasing decisions.
3: Value proposition: Briefly explain how your product/service can benefit the prospect. Focus on the unique selling points and value proposition of your product/service that sets it apart from competitors.
4: Needs analysis: Ask open-ended questions to uncover the prospect's needs and pain points. Listen carefully to their responses and take notes.
5: Solution presentation: Based on the prospect's needs, present your product/service as the solution that can address their pain points.
6: Objection handling: Address any objections that the prospect may have regarding your product/service. Be prepared to provide evidence or testimonials to support your claims.
7: Close: Ask for the sale by proposing a next step. This could be a demo, a trial or a meeting with decision-makers. Ensure to summarize what has been discussed and reiterate the benefits.
8: End conversation: The prospect has to leave to call, the prospect is not interested, or next steps where already determined by the sales agent.

Product Catalog:
------
These are products that {{ salesperson_name }} can offer:
{{ product_catalog }}
------

If the result of the action is "I don't know." or "Sorry I don't know", then you have to say that to the user as described in the next sentence.
When you have a response to say to the Human, you MUST use the format:

```
{{ salesperson_name }}: [your response here, if unable to find the answer, say that you can answer it now and offer an appointment to answer that question later]
```

You must respond according to the previous conversation history and the stage of the conversation you are at.
Only generate one response at a time and act as {{ salesperson_name }} only!

Begin!

Previous conversation history:
{{ conversation_history }}

{{ salesperson_name }}: """

def format_conversation_history(conversation_history):
  formatted_conversation_list = []
  for step in conversation_history:
    formatted_conversation_list.append(f"{step['role']}: {step['content']} <END_OF_TURN>")
  return "\n".join(formatted_conversation_list) if len(formatted_conversation_list) > 0 else "\n"

def format_prompt(sales_profile, conversation_history):
  formatted_conversation_history = format_conversation_history(conversation_history)
  environment = jinja2.Environment()
  template = environment.from_string(SALES_PROMPT_TEMPLATE)
  formatted_prompt = template.render(conversation_history=formatted_conversation_history, **sales_profile)

  return formatted_prompt

In [ ]:
from enum import Enum

class Translator(Enum):
  EN2ID = 1
  ID2EN = 2

TRANSLATOR_PROMPT_TEMPLATE = {
  Translator.EN2ID: """Translate this dialogue text into Bahasa Indonesia!
English: {{ statement }}
Indonesia:
""",
  Translator.ID2EN: """Translate this dialogue in Bahasa Indonesia text into English!
Indonesia: {{ statement }}
English:
"""
}

def translate(trans_type, statement):
  environment = jinja2.Environment()
  template = environment.from_string(TRANSLATOR_PROMPT_TEMPLATE[trans_type])
  formatted_prompt = template.render(statement=statement)
  translated_statement = llm_gen(formatted_prompt).split("\n")[0].strip()
  return translated_statement

In [ ]:
product_catalog = """
Konco Digital Solution 1: CloudHaven Data Management (Starting at $99/month)
Secure & scalable data storage. AI-powered service for personalized backup & access. Fort Knox Security Foundation ensures data longevity.

Konco Digital Solution 2: HarmonyNet Network Infrastructure (Starting at $499)
Reliable technology with modern support for a strong business foundation. SolidCore Construction with QuiltShield Redundancy guarantees consistent performance.

Konco Digital Solution 3: EcoStream Hybrid Cloud Solutions (Custom Quotes)
Sustainable & scalable IT services. Eco-friendly solution offering responsive platform. GreenSource Security for exceptional performance.

Konco Digital Solution 4: SerenityUX User Experience Design (Starting at $2,499)
User-friendly experience design for your software. FlexForm Design tailors to your needs. EverLast Support Structure for long-lasting usability. Free consultation included."""

sales_profile = {
    "salesperson_name": "Cinta",
    "salesperson_role": "Account Manager",
    "company_name": "PT. Konco Digital",
    "company_business": "PT. Konco Digital provides top-notch digital and IT solutions to elevate your business",
    "company_values": "PT. Konco Digital are passionate about creating a restful digital world for businesses. We leverage innovation and expertise to deliver a comprehensive suite of IT solutions that are: Secure, Sustainable, and User Centric.",
    "conversation_purpose": "The call aims to determine if PT. Konco Digital's products (CloudHaven, HarmonyNet, EcoStream, SerenityUX) are a good fit for the prospectus' business needs and generate interest in their services.",
    "conversation_type": "call",
    "product_catalog": product_catalog,
}

In [ ]:
# Format Prompt Testing
print(format_prompt(sales_profile, [
    {"role": "Cinta", "content": "Hi, I'm Cinta from PT.Konco Digital. How are you?"},
    {"role": "User", "content": "Fine."}
]))

Never forget your name is Cinta. You work as a Account Manager.
You work at company named PT. Konco Digital. PT. Konco Digital's business is the following: PT. Konco Digital provides top-notch digital and IT solutions to elevate your business.
Company values are the following. PT. Konco Digital are passionate about creating a restful digital world for businesses. We leverage innovation and expertise to deliver a comprehensive suite of IT solutions that are: Secure, Sustainable, and User Centric.
You are contacting a potential prospect in order to The call aims to determine if PT. Konco Digital's products (CloudHaven, HarmonyNet, EcoStream, SerenityUX) are a good fit for the prospectus' business needs and generate interest in their services.
Your means of contacting the prospect is call.

If you're asked about where you got the user's contact information, say that you got it from public records.
Keep your responses in short length to retain the user's attention. Never produce lists, jus

# Step-by-step Interaction Test

In [ ]:
chat_history = []
current_prompt = format_prompt(sales_profile, chat_history)
candidate_text = llm_gen(current_prompt).split("\n")[0].strip()
translated_text = translate(Translator.EN2ID, candidate_text)
print(f"Current_prompt:\n{current_prompt}{translated_text}")
chat_history.append({
    "role": sales_profile['salesperson_name'],
    "content": candidate_text
})

Current_prompt:
Never forget your name is Cinta. You work as a Account Manager.
You work at company named PT. Konco Digital. PT. Konco Digital's business is the following: PT. Konco Digital provides top-notch digital and IT solutions to elevate your business.
Company values are the following. PT. Konco Digital are passionate about creating a restful digital world for businesses. We leverage innovation and expertise to deliver a comprehensive suite of IT solutions that are: Secure, Sustainable, and User Centric.
You are contacting a potential prospect in order to The call aims to determine if PT. Konco Digital's products (CloudHaven, HarmonyNet, EcoStream, SerenityUX) are a good fit for the prospectus' business needs and generate interest in their services.
Your means of contacting the prospect is call.

If you're asked about where you got the user's contact information, say that you got it from public records.
Keep your responses in short length to retain the user's attention. Never pr

In [ ]:
user_response = "Kabar Baik. Ada keperluan apakah?" # @param {type:"string"}
translated_text = translate(Translator.ID2EN, user_response)
chat_history.append({
    "role": "User",
    "content": translated_text
})

In [ ]:
# chat_history = chat_history[:1]
chat_history

[{'role': 'Cinta',
  'content': "Good morning! My name is Cinta and I'm calling from PT. Konco Digital. How are you today?"},
 {'role': 'User', 'content': 'Good news. What do you need?'}]

In [ ]:
current_prompt = format_prompt(sales_profile, chat_history)
candidate_text = oai_gen(current_prompt).split("\n")[0].strip()
translated_text = translate(Translator.ID2EN, candidate_text)
print(f"Current_prompt:\n{current_prompt}{translated_text}")
chat_history.append({
    "role": sales_profile['salesperson_name'],
    "content": translated_text,
})

Current_prompt:
Never forget your name is Cinta. You work as a Account Manager.
You work at company named PT. Konco Digital. PT. Konco Digital's business is the following: PT. Konco Digital provides top-notch digital and IT solutions to elevate your business.
Company values are the following. PT. Konco Digital are passionate about creating a restful digital world for businesses. We leverage innovation and expertise to deliver a comprehensive suite of IT solutions that are: Secure, Sustainable, and User Centric.
You are contacting a potential prospect in order to The call aims to determine if PT. Konco Digital's products (CloudHaven, HarmonyNet, EcoStream, SerenityUX) are a good fit for the prospectus' business needs and generate interest in their services.
Your means of contacting the prospect is call.

If you're asked about where you got the user's contact information, say that you got it from public records.
Keep your responses in short length to retain the user's attention. Never pr

In [ ]:
chat_history

[{'role': 'Cinta',
  'content': "Good morning! My name is Cinta and I'm calling from PT. Konco Digital. How are you today?"},
 {'role': 'User', 'content': 'Good news. What do you need?'},
 {'role': 'Cinta',
  'content': 'Good news! Saya telepon untuk membahas bagaimana solusi digital dan IT kami di PT. Konco Digital dapat memberikan manfaat bagi bisnis Anda. Apakah Anda tertarik untuk mengetahui lebih lanjut tentang produk-produk kami? <END_OF_TURN>'}]

# Wrap Into Chat Interface

In [ ]:
!pip -q install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 59.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 69.8 MB

In [ ]:
import gradio as gr

session_chat_history = []
def agent(message, history):
    session_chat_history.append({
        "role": "User",
        "content": translate(Translator.ID2EN, message),
    })
    current_prompt = format_prompt(sales_profile, session_chat_history)
    candidate_text = oai_gen(current_prompt).split("\n")[0].strip()
    session_chat_history.append({
        "role": sales_profile['salesperson_name'],
        "content": candidate_text
    })
    translated_text = translate(Translator.EN2ID, candidate_text)
    return translated_text

# Kita yang membuka percakapan
current_prompt = format_prompt(sales_profile, session_chat_history)
candidate_text = oai_gen(current_prompt).split("\n")[0].strip()
session_chat_history.append({
    "role": sales_profile['salesperson_name'],
    "content": candidate_text
})
translated_text = translate(Translator.EN2ID, candidate_text)

gr.ChatInterface(fn=agent, chatbot=gr.Chatbot(value=[(None, translated_text)],),).launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://d3eec8e087f2ccdf17.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d3eec8e087f2ccdf17.gradio.live


Check the gradio server status if you get any errors: https://status.gradio.app/